In [39]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from imblearn.over_sampling import RandomOverSampler 

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [40]:
##loading data

In [41]:
numerical_df = pd.read_csv('files_for_lab/numerical.csv')
categorical_df = pd.read_csv('files_for_lab/categorical.csv')
target_df = pd.read_csv('files_for_lab/target.csv')

In [42]:
df = pd.concat([numerical_df, categorical_df,target_df], axis=1)
df.head(4)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,...,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,...,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,...,91,11,92,7,95,12,90,1,0,0.0
3,0,70.000000,1,4,2,0,23,14,31,3,...,87,11,94,11,95,12,87,2,0,0.0


In [43]:
df.isnull().sum().sum()

0

In [44]:
y = df['TARGET_B']
X = df.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [45]:
df['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [46]:
# for upsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)

from sklearn.utils import resample
category_0 = trainset[trainset['TARGET_B']== 0 ]
category_1 = trainset[trainset['TARGET_B']== 1 ]
category_1 = resample(category_1,
                    replace=True, 
                    n_samples = len(category_0),
                    random_state=0)


In [47]:
display(category_0.shape)
display(category_1.shape)

(72486, 356)

(72486, 356)

In [48]:
trainset_new = pd.concat([category_0, category_1], axis = 0)
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']

In [49]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)



In [50]:
y_train.value_counts()

0    72486
1    72486
Name: TARGET_B, dtype: int64

In [51]:
y_test.value_counts()

0    18083
1     1000
Name: TARGET_B, dtype: int64

In [53]:
clf = RandomForestClassifier(max_depth=3,
                             min_samples_split=10,
                             min_samples_leaf =10,
                             max_samples=0.5,
                             random_state = 42)
clf.fit(X_train, y_train)

print("The Accuracy for the Random Forest in the TRAIN set is {:.2f}".format(clf.score(X_train, y_train)))
print("The Accuracy for the Random Forest in the TEST  set is {:.2f}".format(clf.score(X_test, y_test)))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

C:\Users\pbevi\anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\pbevi\anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


The Accuracy for the Random Forest in the TRAIN set is 0.97
The Accuracy for the Random Forest in the TEST  set is 0.98


C:\Users\pbevi\anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\pbevi\anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[17837,   246],
       [   68,   932]], dtype=int64)

In [ ]:
# I tried multiple types this lab but this is wrong because the other one gave me 0.6, I don´t know what I am missing here.